In [3]:
import pandas as pd

In [4]:
import numpy as np

SimpleImputer. 👉 Purpose: Missing values ko handle karna (fill karna)
Data set me kabhi kabhi NaN (missing values) hote hain. Agar unko directly model me dalenge to model error dega.
SimpleImputer ka kaam hota hai missing values ko fill karna kisi strategy se.Example: mean, median, mode ya koi constant value.

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [8]:
df=pd.read_csv('covid_toy.csv')

In [9]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_test,y_train=train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [13]:
X_train

,age,gender,fever,cough,city
8,19,Female,100.0,Strong,Bangalore
38,49,Female,101.0,Mild,Delhi
68,54,Female,104.0,Strong,Kolkata
51,11,Female,100.0,Strong,Kolkata
55,81,Female,101.0,Mild,Mumbai
...,...,...,...,...,...
26,19,Female,100.0,Mild,Kolkata
1,27,Male,100.0,Mild,Delhi
18,64,Female,98.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore


### Aam Zindagi (manual process)
Tumne preprocessing har feature ko alag-alag handle karke manually transform kiya. Fever column → SimpleImputer

In [18]:
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit_transform(X_test[['fever']])


### 2. Cough column → OrdinalEncoder 
Need: cough ek ordered categorical variable hai → "Mild" < "Strong".OrdinalEncoder ne Mild = 0, Strong = 1 bana diya.

In [23]:
oe=OrdinalEncoder(categories=[['Mild','Strong']])

In [48]:
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.fit_transform(X_test[['cough']])

### 3. Gender + City → OneHotEncoder

Need: Gender aur City nominal categorical variables hain (order nahi hai).OneHotEncoder ne unhe dummy variables (0/1) me convert kar diya.
drop='first' → dummy variable trap avoid karne ke liye ek column drop kiya.

In [31]:
ohe=OneHotEncoder(drop='first',sparse_output=False)

('tnf1', SimpleImputer(), ['fever']) → fever column me missing values fill.
('tnf2', OrdinalEncoder(...), ['cough']) → cough ko ordinal encode.
('tnf3', OneHotEncoder(...), ['gender','city']) → gender aur city ko one-hot encode.
remainder='passthrough' → baaki jo columns (age) bache hain unhe as-is copy kar lena.

In [34]:
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])

In [36]:
X_test_gender_city=ohe.fit_transform(X_test[['gender','city']])

In [38]:
## Extracting Age Need: Age pehle hi numeric hai → direct use kar sakte hain.
X_train_age=X_train.drop(columns=['age','gender','fever','cough','city']) 

In [40]:
X_test_age=X_test.drop(columns=['age','gender','fever','cough','city'])

In [44]:
#Need: Saare transformed features ko ek final matrix me combine karna. Shape (80,7) = Matlab final training data me 7 features ban gaye.
X_train_tranformed=np.concatenate((X_train_age,X_train_cough,X_train_gender_city,X_train_fever),axis=1)

In [50]:
X_test_tranformed=np.concatenate((X_test_age,X_test_cough,X_test_gender_city,X_test_fever),axis=1) 

### Mentos Zindagi (using ColumnTransformer)
Yaha tumne wahi kaam kiya but ek hi line me, automatically ✅

In [53]:
from sklearn.compose import ColumnTransformer

In [59]:
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),[['gender','city']])
],remainder='passthrough')

In [65]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender','city'])
], remainder='passthrough')

transformer.fit_transform(X_train).shape

(80, 7)